# actor critic (part2: 训练agent)

In [1]:
# 环境配置
%cd /playground/sgd_deep_learning/sgd_rl/go
import sys
sys.path.append('./python')

/playground/sgd_deep_learning/sgd_rl


In [2]:
import os
import time
import torch

from dlgo import agent
from dlgo import rl
from dlgo.encoders import get_encoder_by_name
from dlgo.networks import acnet_small

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 数据 模型存放目录
data_home_path = 'data/ac/'
if not os.path.exists(data_home_path):
    os.makedirs(data_home_path)

class args:
    learning_agent= data_home_path + 'agent_checkpoint.pth'
    experience = [data_home_path + 'experience.pth'] # List
    agent_out = data_home_path + 'agent_checkpoint_update.pth'
    
    board_size = 9 # 缩小计算量, 保证算法的验证速度
    num_games = 10 # 每轮迭代只收集10games的数据
    lr=0.01 # 学习率
    batch_size =512 # batch_size
    
# 全局变量
global BOARD_SIZE
BOARD_SIZE = args.board_size

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("model file exists: ", os.path.exists(args.learning_agent))
print("device:", device)

model file exists:  False
device: cuda


In [4]:
def main():
    learning_agent_filename = args.learning_agent
    updated_agent_filename = args.agent_out
    experience_files = args.experience
    learning_rate = args.lr
    batch_size = args.batch_size
    
    # init agent object
    encoder_name = 'sevenplane'
    model = acnet_small(input_channel_num=7, board_size=BOARD_SIZE)

    # 训练阶段只需要一个agent, train self.model 即可
    learning_agent = None
    if not os.path.exists(learning_agent_filename): # check_point不存在
        encoder = get_encoder_by_name(name=encoder_name, board_size=BOARD_SIZE)
        learning_agent = rl.load_ac_agent(model=model, encoder=encoder, device=device)
    else:
        learning_agent = rl.load_ac_agent(model=model, save_path=learning_agent_filename, device=device)
    assert learning_agent is not None

    # 读取数据训练
    for exp_filename in experience_files:
        exp_buffer = rl.load_experience(exp_filename)
        
        learning_agent.train(
            exp_buffer,
            lr=learning_rate,
            batch_size=batch_size)

    # 更新后新模型保存
    learning_agent.serialize(updated_agent_filename)

main()

iter: 1 batch_size 512
iter: 2 batch_size 512
iter: 3 batch_size 144
